In [ ]:
using MLBase
using Flux
using Flux: onehotbatch, argmax, crossentropy, throttle
using Plots
using DecisionTree
using Distances
using LIBSVM
using CSV
using DataFrames
using Random

In [ ]:
iris = Iris()
iris_m = Matrix(iris.dataframe)
iris_m = iris_m[shuffle(1:end), :]
train_test_ratio = .7
idx = Int(floor(size(iris_m, 1) * train_test_ratio))
data_train  = iris_m[1:idx,:]
data_test = iris_m[idx+1:end, :]
x_train = transpose(Float32.(data_train[:,1:4]))
x_test = transpose(Float32.(data_test[:,1:4]))
findaccuracy(predictedvals, groundtruthvals) = sum(predictedvals.==groundtruthvals)/length(groundtruthvals)
iris_m

In [ ]:
y_train = onehotbatch(data_train[:,5],unique(iris_m[:,5]))
y_test = onehotbatch(data_test[:,5], unique(iris_m[:,5]))

In [ ]:
model_DT = DecisionTreeClassifier(max_depth=4)
DecisionTree.fit!(model_DT, x_train', data_train[:,5])
pred_DT = DecisionTree.predict(model_DT,x_test')
DT=findaccuracy(pred_DT,data_test[:,5])

In [ ]:
data_train = [(x_train, y_train)]
data_test = [(x_test, y_test)]
#train_d1 = DataLoader((x_train, y_train), batchsize=batch_size, shuffle=true)

In [ ]:
function get_model()
    c = Chain(
        Dense(4,8,relu),
        Dense(8,8,relu),
        Dense(8,3),
        softmax
    )
end

model = get_model()

loss(x,y) = crossentropy(model(x),y)
opt = ADAM()
evalcb = () -> @show(loss(x_train, y_train))
parameters = Flux.params(model)

In [ ]:
for i in 1:600
    Flux.train!(loss, parameters, data_train, opt, cb = throttle(evalcb, 10))
end

In [ ]:
y = (y_test[:,25])

In [ ]:

pred = (model(x_test[:,25]))

In [ ]:
function loss_all(data_loader)
    sum([loss(x, y) for (x,y) in data_loader]) / length(data_loader) 
end

function acc(data_loader)
    f(x) = Flux.onecold(cpu(x))
    acces = [sum(f(model(x)) .== f(y)) / size(x,2)  for (x,y) in data_loader]
    sum(acces) / length(data_loader)
end

In [ ]:
data_test
@show train_loss = loss_all(data_t)
@show test_loss = loss_all(data_test)
@show train_acc = acc(data_t)
@show test_acc = acc(data_test)
